In [1]:
# Main notebook for estimating Rt using the method from the Aarhus Datathon
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget  
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [3]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData/ssi_data\SSI_data_2021-08-20
2021-08-20 00:00:00


In [12]:
filename = 'Newly_admitted_over_time.csv'
df = pd.read_csv(latestdir+'\\'+filename,sep=';')
df['Dato'] = pd.to_datetime(df.Dato)
df.tail()


,Dato,Hovedstaden,Sjælland,Syddanmark,Midtjylland,Nordjylland,Ukendt Region,Total
532,2021-08-15,13,1,6,7,2,0,29
533,2021-08-16,12,4,6,4,3,1,30
534,2021-08-17,10,3,5,6,2,0,26
535,2021-08-18,15,8,6,3,1,0,33
536,2021-08-19,14,4,5,5,4,1,33


In [24]:
# Estimate country-wide, daily

curAdm = df.Total.values

tSerial = 4.7
serialFactor = tSerial/7

hospRatio = curAdm[:-7]/curAdm[7:]
curT = df.Dato.values[7:]

fig,ax1 = plt.subplots()

ax1.plot(curT,hospRatio,'x-',color='grey',linewidth=0.5,markersize=2)
ax1.plot(rnTime(curT,7),rnMean(hospRatio,7),'k')
plt.tight_layout()

<ipython-input-24-c8962ffe8da2>:8: RuntimeWarning: divide by zero encountered in true_divide
  hospRatio = curAdm[:-7]/curAdm[7:]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
# Make weekly sums, and use those instead

admWeek = rnMean(curAdm,7)[::7]
tWeek = curT[::7]

hospRatioWeek = admWeek[:-1]/admWeek[1:] 
curTW = tWeek[1:]

fig,ax1 = plt.subplots()

# ax1.plot(rnTime(curTW,7),rnMean(hospRatioWeek,7),'b')
ax1.step(rnTime(curT,7),rnMean(hospRatio,7),'k')
ax1.step(curTW,hospRatioWeek,'b--')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …